<a href="https://colab.research.google.com/github/YinGuoX/Deep_Learning_Pytorch_WithDeeplizard/blob/master/32_Training_Loop_Run_Builder_Neural_Network_Experimentation_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Loop Run Builder - Neural Network Experimentation
在本集中，我们将编写一个RunBuilder类，该类将允许我们使用不同的参数生成多个运行。

## 1.使用RunBuilder类

本节以及本系列最后几节的目的是使自己处于能够有效地尝试我们构建的训练过程的位置。 因此，我们将扩展在超参数实验的情节中所涉及的内容。 我们将使那里看到的更加干净。

我们将构建一个名为RunBuilder的类。 但是，在我们探讨如何构建类之前。 让我们看看它将允许我们做什么。 我们将从import开始。


In [ ]:
from collections import OrderedDict
from collections import namedtuple
from itertools import product

我们从集合中导入OrderedDict和namedtuple，并从itertools中导入一个名为product的函数。 这个product（）函数是我们上次看到的函数，它在给定多个列表输入的情况下计算笛卡尔乘积。

好吧。 这是RunBuilder类，它将构建用于定义运行的参数集。 在看到如何使用它之后，我们将看到它的工作原理。

In [ ]:
class RunBuilder():
  @staticmethod
  def get_runs(params):
    Run = namedtuple("Run",params.keys())

    runs = []

    for v in product(*params.values()):
      runs.append(Run(*v))
    
    return runs

关于使用此类的主要注意事项是它具有一个称为get_runs（）的静态方法。 该方法将为我们提供基于传入参数构建的运行结果。

现在定义一些参数。

In [ ]:
params = OrderedDict(
    lr=[0.01,0.001],
    batch_size = [1000,10000]
)

在这里，我们在字典中定义了一组参数和值。 我们有一组学习率和一组批次大小，我们想尝试一下。 当我们说“尝试”时，是指我们要针对字典中的每个学习率和每个批次大小进行一次训练。

要获得这些运行，我们只需调用RunBuilder类的get_runs（）函数，并传入我们要使用的参数即可。

In [ ]:
runs = RunBuilder.get_runs(params)
runs

[Run(lr=0.01, batch_size=1000),
 Run(lr=0.01, batch_size=10000),
 Run(lr=0.001, batch_size=1000),
 Run(lr=0.001, batch_size=10000)]

很好，我们可以看到RunBuilder类已经构建并返回了四个运行的列表。 这些运行中的每一个都有学习率和定义运行的批处理大小。

我们可以通过索引到列表来访问单个运行，如下所示：

In [ ]:
run = runs[0]
run

Run(lr=0.01, batch_size=1000)

请注意run输出的字符串表示形式。这个字符串表示是由Run tuple类自动生成的，如果我们想将运行统计信息写入TensorBoard或任何其他可视化程序的磁盘，可以使用这个字符串唯一地标识运行。

此外，因为run 是对象是一个具有命名属性的元组，所以我们可以使用点表示法访问值，如下所示：

In [ ]:
print(run.lr,run.batch_size)

0.01 1000


最后，由于运行列表是一个Python iterable，我们可以像这样清晰地迭代运行：

In [ ]:
for run in runs:
  print(run,run.lr,run.batch_size)

Run(lr=0.01, batch_size=1000) 0.01 1000
Run(lr=0.01, batch_size=10000) 0.01 10000
Run(lr=0.001, batch_size=1000) 0.001 1000
Run(lr=0.001, batch_size=10000) 0.001 10000


要添加其他值，我们要做的就是将它们添加到原始参数列表中，如果要添加其他类型的参数，我们要做的就是添加它。 新参数及其值将自动变为可在运行中使用。 运行的字符串输出也将更新。

两个参数：

In [ ]:
params = OrderedDict(
    lr = [.01, .001]
    ,batch_size = [1000, 10000]
)

In [ ]:

runs = RunBuilder.get_runs(params)
runs

[Run(lr=0.01, batch_size=1000),
 Run(lr=0.01, batch_size=10000),
 Run(lr=0.001, batch_size=1000),
 Run(lr=0.001, batch_size=10000)]

In [ ]:
params = OrderedDict(
    lr = [.01, .001]
    ,batch_size = [1000, 10000]
    ,device = ["cuda", "cpu"]
)

In [ ]:

runs = RunBuilder.get_runs(params)
runs

[Run(lr=0.01, batch_size=1000, device='cuda'),
 Run(lr=0.01, batch_size=1000, device='cpu'),
 Run(lr=0.01, batch_size=10000, device='cuda'),
 Run(lr=0.01, batch_size=10000, device='cpu'),
 Run(lr=0.001, batch_size=1000, device='cuda'),
 Run(lr=0.001, batch_size=1000, device='cpu'),
 Run(lr=0.001, batch_size=10000, device='cuda'),
 Run(lr=0.001, batch_size=10000, device='cpu')]

当我们在训练过程中尝试不同的值时，此功能将使我们能够更好地控制。

让我们看看如何构建此RunBuilder类。

## 2.编码RunBuilder类
我们首先需要的是一个字典的参数和值，我们想尝试


In [ ]:
params = OrderedDict(
    lr = [.01, .001]
    ,batch_size = [1000, 10000]
)

然后我们可以获得该字典的key和values

In [ ]:
params.keys()

odict_keys(['lr', 'batch_size'])

In [ ]:
params.values()

odict_values([[0.01, 0.001], [1000, 10000]])

一旦拥有了这两者，我们只需通过检查它们的输出来确保我们对它们都有了解。 完成后，我们将使用这些键和值进行下一步操作。 我们将从键开始。

In [ ]:
Run = namedtuple("Run",params.keys())
print(Run.batch_size)

该行创建了一个名为Run的新元组子类，该子类具有命名字段。 这个Run类用于封装每次运行的数据。 此类的字段名称由传递给构造函数的名称列表设置。 首先，我们传递类名。 然后，我们传递字段名，在本例中，我们传递字典中的键列表。

现在我们有了一个用于运行的类，我们准备创建一些类。

In [ ]:
runs = []

for v in product(*params.values()):
  runs.append(Run(*v))

首先，我们创建一个名为runs的列表。然后，我们使用itertools中的product（）函数，使用字典中每个参数的值创建笛卡尔积。这为我们提供了一组定义运行的有序对。我们迭代这些过程，为每个运行添加一个运行到运行列表中。

对于笛卡尔积中的每个值，我们都有一个有序元组。笛卡尔积给出了每一个有序对，所以我们有学习率和批量大小的所有可能的有序对。当我们将元组传递给Run构造函数时，我们使用*操作符告诉构造函数接受元组值作为参数，而不是元组本身。

最后，我们将此代码包装到RunBuilder类中。

In [ ]:
class RunBuilder():
    @staticmethod
    def get_runs(params):

        Run = namedtuple('Run', params.keys())

        runs = []
        for v in product(*params.values()):
          runs.append(Run(*v))
        return runs

由于get_runs（）方法是静态的，因此我们可以使用类本身来调用它。 我们不需要该类的实例。

现在，这使我们可以通过以下方式更新我们的培训代码：

前：



```
for lr, batch_size, shuffle in product(*param_values):
    comment = f' batch_size={batch_size} lr={lr} shuffle={shuffle}'

    # Training process given the set of parameters
```

后：



```
for run in RunBuilder.get_runs(params):
    comment = f'-{run}'

    # Training process given the set of parameters
```




## 3.解释什么是笛卡尔乘积？
百度好吧


代码实现

In [ ]:
X = {1,2,3}
Y = {1,2,3}

{(x,y) for x in X for y in Y}

{(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)}

注意数学代码的功能。 它涵盖了所有情况。 也许您注意到，可以使用如下的for循环迭代来实现：

In [ ]:
X = {1,2,3}
Y = {5,6,7}

cartesian_product =set()
for x in X:
  for y in Y:
    cartesian_product.add((x,y))

cartesian_product

{(1, 5), (1, 6), (1, 7), (2, 5), (2, 6), (2, 7), (3, 5), (3, 6), (3, 7)}

好了，现在我们知道了它是如何工作的，并且可以继续使用它。